In [1]:
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, StringType
# import json

# # Create Spark Session
# # spark = SparkSession.builder \
# #     .appName("MCC_Loader") \
# #     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
# #     .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
# #     .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
# #     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
# #     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
# #     .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
# #     .config("spark.jars.packages", 
# #             "org.apache.hadoop:hadoop-aws:3.3.4,"
# #             "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
# #             "org.postgresql:postgresql:42.6.0") \
# #     .getOrCreate()

# spark = SparkSession.builder \
#     .appName("MCC_Loader") \
#     .master("spark://spark-master:7077") \
#     .config("spark.executor.memory", "1g") \
#     .config("spark.executor.cores", "1") \
#     .config("spark.cores.max", "3") \
#     .config("spark.sql.adaptive.enabled", "true") \
#     .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
#     .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
#     .config("spark.jars.packages", 
#             "org.apache.hadoop:hadoop-aws:3.3.4,"
#             "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
#             "org.postgresql:postgresql:42.5.4") \
#     .getOrCreate()

# # Read JSON file from MinIO
# #s3a://dev/users_data.csv
# json_rdd = spark.sparkContext.textFile("s3a://dev/mcc_codes.json")
# json_str = json_rdd.collect()[0]  # Get the entire JSON as string
# mcc_dict = json.loads(json_str)

# # Convert dict to list of tuples
# mcc_data = [(code, desc) for code, desc in mcc_dict.items()]

# # Define schema
# schema = StructType([
#     StructField("mcc_code", StringType(), False),
#     StructField("description", StringType(), False)
# ])

# # Create DataFrame
# df = spark.createDataFrame(mcc_data, schema=schema)

# # Show data
# print("Data to be loaded:")
# df.show(truncate=False)

# # PostgreSQL properties
# postgres_props = {
#     "user": "postgres",
#     "password": "postgres",
#     "driver": "org.postgresql.Driver"
# }

# # Load to PostgreSQL
# df.write.jdbc(
#     url="jdbc:postgresql://postgres:5432/gold_db",
#     table="mcc_codes",
#     mode="append",
#     properties=postgres_props
# )

# print(f"Loaded {df.count()} records successfully!")
# spark.stop()

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import json
spark = SparkSession.builder \
    .appName("MCC_Loader") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.jars.packages", 
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
            "org.postgresql:postgresql:42.5.4") \
    .getOrCreate()

# Read JSON file from MinIO
#s3a://dev/users_data.csv
# df_mcc_codes = spark.read.option("multiline", "true").json("s3a://dev/mcc_codes.json")
# df_mcc_codes.show(10, truncate=False)

# json_rdd = spark.sparkContext.textFile("s3a://dev/mcc_codes.json")

# Read the JSON file as a single string
raw_rdd = spark.sparkContext.wholeTextFiles("s3a://dev/mcc_codes.json").map(lambda x: x[1])

# Parse JSON string into Python dict
parsed_rdd = raw_rdd.map(lambda x: json.loads(x))

# Flatten dict into (key, value) pairs
flat_rdd = parsed_rdd.flatMap(lambda d: d.items())

# Convert to DataFrame
df = flat_rdd.toDF(["mcc_code", "description"])

df.show(truncate=False)

# PostgreSQL properties
postgres_props = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

# Load to PostgreSQL
df.write.jdbc(
    url="jdbc:postgresql://postgres:5432/gold_db",
    table="mcc_codes",
    mode="append",
    properties=postgres_props
)

print(f"Loaded {df.count()} records successfully!")
spark.stop()


+--------+------------------------------------------+
|mcc_code|description                               |
+--------+------------------------------------------+
|5812    |Eating Places and Restaurants             |
|5541    |Service Stations                          |
|7996    |Amusement Parks, Carnivals, Circuses      |
|5411    |Grocery Stores, Supermarkets              |
|4784    |Tolls and Bridge Fees                     |
|4900    |Utilities - Electric, Gas, Water, Sanitary|
|5942    |Book Stores                               |
|5814    |Fast Food Restaurants                     |
|4829    |Money Transfer                            |
|5311    |Department Stores                         |
|5211    |Lumber and Building Materials             |
|5310    |Discount Stores                           |
|3780    |Computer Network Services                 |
|5499    |Miscellaneous Food Stores                 |
|4121    |Taxicabs and Limousines                   |
|5300    |Wholesale Clubs   